In [1]:
# import the necessary libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load data
df_transformed = pd.read_csv(".data/processed/data-population-victims-of-armed-conflict.csv")

# Analize df
df_transformed.head(5)

# Load

## Creation of DW

In [ ]:
# Create database for DW
conn_dw = sqlite3.connect(ruta_base + "Project_ETL.db")
cursor_dw = conn_dw.cursor()

cursor_dw.executescript("""

-- DESACTIVAR CLAVES FORÁNEAS
PRAGMA foreign_keys = OFF;


-- ELIMINAR TABLAS (orden correcto)
DROP TABLE IF EXISTS victims;
DROP TABLE IF EXISTS person;
DROP TABLE IF EXISTS victimizing_act;
DROP TABLE IF EXISTS location;
DROP TABLE IF EXISTS registration_date;

-- ACTIVAR CLAVES FORÁNEAS
PRAGMA foreign_keys = ON;


-- CREAR DIMENSIONES
CREATE TABLE person (
    id_person INTEGER PRIMARY KEY NOT NULL,
    sex TEXT CHECK (sex IN ('Masculino', 'Femenino')),
    ethnic_group TEXT,
    classification TEXT
);

CREATE TABLE victimizing_act (
    id_act INTEGER PRIMARY KEY NOT NULL,
    victimization_fact TEXT
);

CREATE TABLE location (
    id_location INTEGER PRIMARY KEY NOT NULL,
    commune TEXT
);

CREATE TABLE registration_date (
    id_date INTEGER PRIMARY KEY NOT NULL,
    date_processing DATE,
    year INTEGER
);

CREATE TABLE victims (
    id_person INTEGER,
    id_act INTEGER,
    id_location INTEGER,
    id_date INTEGER,
    vulnerability_index INTEGER,
    total_victim INTEGER,
    FOREIGN KEY (id_person) REFERENCES person(id_person),
    FOREIGN KEY (id_act) REFERENCES victimizing_act(id_act),
    FOREIGN KEY (id_location) REFERENCES location(id_location),
    FOREIGN KEY (id_date) REFERENCES registration_date(id_date)
);""")

## separate column for data entry

In [ ]:
# person column data
people = df_transformed[["sex", "ethnic_group","classification"]].drop_duplicates()
people["id_person"] = people.index + 1
people.head(5)

In [ ]:
# victimizing_act column data
victimizing_act = df_transformed[["victimization_fact"]].drop_duplicates().reset_index(drop=True)
victimizing_act["id_act"] = victimizing_act.index + 1
victimizing_act.head(15)

In [ ]:
# registration_date column data
registration_date = df_transformed[["date_processing"]].drop_duplicates().copy()
registration_date["date_processing"] = pd.to_datetime(registration_date["date_processing"])
registration_date["year"] = registration_date["date_processing"].dt.year
registration_date["id_date"] = registration_date.index + 1
registration_date.head(10)

In [ ]:
# location column data
location =  df_transformed[["commune"]].drop_duplicates()
location["id_location"] = location.index + 1
location.head(23)

In [ ]:
# Load data into the DW
people.to_sql("person", conn_dw, if_exists="append", index=False)
victimizing_act.to_sql("victimizing_act", conn_dw, if_exists="append", index=False)
location.to_sql("location", conn_dw, if_exists="append", index=False)
registration_date.to_sql("registration_date", conn_dw, if_exists="append", index=False)

In [ ]:
# Function to display a table
def display_table(name_table):
  df_table = pd.read_sql(f"SELECT * FROM {name_table}", conn_dw)
  print(f"Tabla: {name_table}")
  display(df_table)


# Visualizar cada tabla del modelo dimensional
display_table("person")
display_table("victimizing_act")
display_table("location")
display_table("registration_date")

In [ ]:
# Copiar dataframe original
fact = df_transformed.copy()

#  The two columns must have the same type in both dataframes
fact["date_processing"] = pd.to_datetime(fact["date_processing"])
registration_date["date_processing"] = pd.to_datetime(registration_date["date_processing"])


# Merge with person
fact = fact.merge(
    people[["id_person","sex","ethnic_group","classification"]],
    on=["sex","ethnic_group","classification"],
    how="left"
)


# Merge with victimizing_act
fact = fact.merge(
    victimizing_act[["id_act","victimization_fact"]],
    on="victimization_fact",
    how="left"
)


# Merge with location
fact = fact.merge(
    location[["id_location","commune"]],
    on="commune",
    how="left"
)


# Merge with date
fact = fact.merge(
    registration_date[["id_date","date_processing"]],
    on="date_processing",
    how="left"
)

# remove descriptive columns
fact = fact.drop(columns=[
    "sex",
    "ethnic_group",
    "vulnerability_level",
    "victimization_fact",
    "commune",
    "date_processing"
])

In [ ]:
# Copiar dataframe original
fact = df_transformed.copy()

#  The two columns must have the same type in both dataframes
fact["date_processing"] = pd.to_datetime(fact["date_processing"])
registration_date["date_processing"] = pd.to_datetime(registration_date["date_processing"])


# Merge with person
fact = fact.merge(
    people[["id_person","sex","ethnic_group","classification"]],
    on=["sex","ethnic_group","classification"],
    how="left"
)


# Merge with victimizing_act
fact = fact.merge(
    victimizing_act[["id_act","victimization_fact"]],
    on="victimization_fact",
    how="left"
)


# Merge with location
fact = fact.merge(
    location[["id_location","commune"]],
    on="commune",
    how="left"
)


# Merge with date
fact = fact.merge(
    registration_date[["id_date","date_processing"]],
    on="date_processing",
    how="left"
)

# remove descriptive columns
fact = fact.drop(columns=[
    "sex",
    "ethnic_group",
    "vulnerability_level",
    "victimization_fact",
    "commune",
    "date_processing"
])

In [ ]:
# Load data to fact table
fact_table.to_sql("victims", conn_dw, if_exists="append", index=False)

In [ ]:
display_table("victims")

# Visualization

## Total victims by gender over the years in Cali

In [ ]:
# Connection to the database
conn_dw = sqlite3.connect(ruta_base + "Project_ETL.db")

# obtain victims by gender and year
query = """
SELECT
    rd.year,
    p.sex,
    COUNT(*) as total_victims
FROM victims v
JOIN person p ON v.id_person = p.id_person
JOIN registration_date rd ON v.id_date = rd.id_date
GROUP BY rd.year, p.sex
ORDER BY rd.year;
"""

df_chart = pd.read_sql(query, conn_dw)

# Pivot for multi-line format
df_pivot = df_chart.pivot(
    index="year",
    columns="sex",
    values="total_victims"
)

# Create chart
plt.figure()
df_pivot.plot()

plt.title("Victims Registered by Sex Over the Years")
plt.xlabel("Year")
plt.ylabel("Number of Victims")
plt.legend(title="Sex")
plt.grid(True)

plt.show()

## Total number of victims by ethnic group processed

In [ ]:
# Conexión a la base de datos
conn_dw = sqlite3.connect(ruta_base + "Project_ETL.db")

# Query para obtener total de víctimas por grupo étnico
query = """
SELECT
    p.ethnic_group,
    COUNT(*) as total_victims
FROM victims v
JOIN person p ON v.id_person = p.id_person
GROUP BY p.ethnic_group
ORDER BY total_victims DESC;
"""

df_chart = pd.read_sql(query, conn_dw)

# Crear gráfico de barras horizontales
plt.figure()
plt.barh(df_chart["ethnic_group"], df_chart["total_victims"])

plt.title("Total Victims by Ethnic Group")
plt.xlabel("Number of Victims")
plt.ylabel("Ethnic Group")

plt.gca().invert_yaxis()  # Para que el mayor quede arriba
plt.show()

## What is the most common victimization by municipality in Cali?

In [ ]:
# Agrupar por comuna y hecho
grouped = (
    df_transformed
    .groupby(['commune', 'victimization_fact'])
    .size()
    .reset_index(name='total_victims')
)

# Obtener el hecho más común por comuna
idx = grouped.groupby('commune')['total_victims'].idxmax()
most_common = grouped.loc[idx].reset_index(drop=True)

# Ordenar por número de víctimas
most_common = most_common.sort_values(by='total_victims', ascending=True)

# Crear etiqueta combinada (Comuna - Hecho)
most_common['label'] = (
    "Comuna " + most_common['commune'].astype(str) +
    " - " + most_common['victimization_fact']
)

#  Gráfico horizontal
plt.figure(figsize=(10, 8))
plt.barh(most_common['label'], most_common['total_victims'])

plt.xlabel("Number of Victims")
plt.title("Most Common Victimizing Act by Commune")

plt.tight_layout()
plt.show()

## Year in which the highest number of victims were processed.

In [ ]:
# Conectar a la DW
conn = sqlite3.connect(ruta_base + "Project_ETL.db")

# Query para contar víctimas por año
query = """
SELECT rd.year, COUNT(*) AS total_victims
FROM victims v
JOIN registration_date rd ON v.id_date = rd.id_date
GROUP BY rd.year
ORDER BY rd.year;
"""

victims_per_year = pd.read_sql(query, conn)

conn.close()

#  Gráfico
plt.figure(figsize=(10,6))
plt.bar(victims_per_year['year'], victims_per_year['total_victims'])

plt.xlabel("Year")
plt.ylabel("Number of Victims")
plt.title("Number of Victims Registered per Year")

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()